In [1]:
%matplotlib
#%matplotlib inline
import os
import csv
import numpy as np
import re
import datetime
import fnmatch
from pathlib import Path
from itertools import groupby
import copy
import itertools
import distance
import nltk
#nltk.download('stopwords')
import pandas as pd
import matplotlib.pyplot as plt
import pickle

from nltk.stem import SnowballStemmer
from nltk.stem import snowball

from itertools import *
from operator import *

# import other jupyter notebooks
import import_ipynb
from openpyxl import load_workbook

Using matplotlib backend: Qt5Agg


In [37]:
# exceptional removal of particular extra sentences not typed by the user 
dict_phraseStim = {
    '2019-01-30-11-22-25_1' : [4, 6, 7], # ys, session 4
    '2019-01-31-09-37-5_2ndPart_2' : range(1,5), # bh1, session 4 , all sentences except the first one deleted
    #'2019-02-05-14-10-39_2ndPart_2' : [1, 2, 3, 4, 5, 6, 9, 10],
    #'2019-01-14-14-58-30' : [0], # ys, session_trial ()
    '2019-01-16-16-36-17_1stPart_2' : [-1], # af_session1
    '2019-01-16-17-00-12_2ndPart_2': [1], # af_session1
    '2019-01-17-15-27-20_1stPart_2' : [4], # Af session2
    '2019-01-17-16-03-27_2ndPart_2' : [0, 1, 2], # Af session2
    '2019-02-06-11-25-41_1' : [7],               # aq_session1    
    '2019-02-08-11-33-53_1stPart_1' : [1],  # aq session3_1_part1
    '2019-02-08-12-11-34_2ndPart_1' : [0, 1, 2, 3],  # aq session3_1_part2
    '2019-01-31-09-22-49_1stPart_2' : [4],  # bh1_session4_2_part1
    '2019-02-14-13-28-20_1stPart_2' : [2], # ch_session3_2_part1
    '2019-02-14-13-57-41_2ndPart_2' : [0, 2, 3], # ch_session3_2_part2
    '2019-01-14-15-07-21_1' : [4], # ys_session1
    '2019-01-16-15-18-50_1stPart_1' : [3, 4], # ys_session2
    '2019-01-16-15-42-51_2ndPart_1' : [2], # ys_session2
    '2019-01-30-11-22-25_1' : [3, 5, 7],          # ys_session4
    '2019-01-16-15-18-0_1' : [4],            # no_session1
    '2019-02-21-16-09-44_1stPart_1' : [1], # bh2_session1
    '2019-02-21-16-22-22_2ndPart_1' : [2, 3, 4],# bh2_session1
    '2019-02-28-17-03-53_1stPart_2' : [2],       # bh2_session3
    '2019-02-28-17-24-2_2ndPart_2' : [0, 2],     # bh2_session3
    '2019-02-21-15-01-4_1stPart_1' : [0],        # le_session3
    '2019-02-21-15-25-56_2ndPart_1' : [1],        # le_session3
    '2019-02-18-10-28-35_2' : [0],               # ls2_session4
    '2019-02-05-14-00-27_1stPart_2' : [3],        # mh_session1
    '2019-02-05-14-10-39_2ndPart_2' : [0, 1, 3],   # mh_session1
    '2019-02-08-10-51-3_1stPart_1' : [4],        # mn_session1
    '2019-02-08-11-05-7_2ndPart_1' : [0, 2, 3, 4], # mn_session1
    '2019-02-19-10-34-7_1stPart_1' : [3],          # mn_session3
    '2019-02-19-10-56-43_2ndPart_1' : [1, 2, 3, 4], # mn_session3
    '2019-01-29-13-25-4_1' : [3],        # ph_session2
    '2019-03-07-16-44-5_2' : [1],                   # rh_session1
    '2019-03-14-13-56-56_2' : [2],                  # rh_session3
    '2019-02-19-17-10-45_1' : [3]                  # ph_session5
}

# exceptional removal of sentences/words typed by the user, but then deleted everything to have a blank scratchpad

dict_phraseUser = {
    "2019-02-06-15-44-15_1" : [2, 3, 6], 
    "2019-02-06-16-19-9_2" : [1, 3, 6, 7],
    "2019-02-12-11-21-21_2" : [0],
    "2019-02-14-14-28-49_1" : [0, 2, 3], # ac_session3_1
    "2019-02-14-14-45-49_2" : [0, 5, 6], # ac_session3_2
    '2019-01-29-14-19-26_1' : [0, 3, 4], # bh1_session2_1
    '2019-01-29-14-40-36_2' : [0, 1, 2], # bh1_session2_2
    '2019-01-30-14-29-29_2' : [4],       # bh1_session3_2
    '2019-01-31-09-12-2_1' : [3],         # bh1_session4_1
    '2019-01-31-09-22-49_1stPart_2' : [4], # bh1_session4_2_part1
    '2019-03-05-09-15-11_1' : [1],         # bh2_session5_1
    '2019-03-05-09-15-11_2' : [1],        # bh2_session5_2
    '2019-02-21-15-55-56_2' : [2],       # ch_session5_2
    '2019-01-30-15-19-36_2' : [1],       # jm_session2_1
    '2019-01-30-15-04-30_1' : [0],         # jm_session2_2
    '2019-01-16-15-18-50_1stPart_1' : [1],  # ys_session2
    '2019-01-16-15-42-51_2ndPart_1' : [0], # ys_session2
    '2019-01-30-11-22-25_1' : [2, 4],       # ys_session4
    '2019-01-30-11-57-3_2' : [0] ,          # ys_session4
    '2019-01-31-13-13-2_1' : [4],           # ys_session5
    '2019-01-30-10-20-32_1' : [0, 1, 2, 3, 4, 5], # no_session4
    '2019-01-30-10-46-38_2' : [0],          # 
    '2019-02-28-17-03-53_1stPart_2' : [2],   # bh2_session3
    '2019-03-12-09-30-5_1' : [0],            # kj_session3
    '2019-02-13-15-20-38_1' : [0, 1, 2, 3, 6], # ls1_session3
    '2019-02-18-10-25-52_1' : [1],              # ls2_session4
    '2019-02-18-10-46-26_2' : [0],            # ls2_session4
    '2019-01-29-13-25-4_1' : [0, 1, 7],        # ph_session2
    '2019-01-29-13-43-50_2' : [0],              # ph_session2
    '2019-03-07-16-17-30_1' : [0],              # rh_session1
    '2019-03-07-16-44-5_2' : [0, 1],         # rh_session1
    '2019-03-14-13-56-56_2' : [0, 1, 3]         # rh_session3
}

# key selection can have extra selections of NextPhrase at the end
dict_keySelectionOfNextPhrase = {
    "2019-02-11-11-18-30_1" : [12, 13], # ac_session1
    "2019-01-16-17-00-12_2ndPart_2" : [12], # af_session1
    "2019-01-17-15-27-20_1stPart_2" : [12], # af_session2
    "2019-02-06-16-19-9_2" : [12], # af_session3
    "2019-02-12-11-07-43_1" : [12], # af_session4
    "2019-02-27-15-08-32_1" : [12], # af_session5
    "2019-01-28-14-30-44_1" : [12], # bh1_session1
    "2019-02-21-16-22-22_2ndPart_1" : [12], # bh2_session1
    "2019-02-18-14-02-56_2" : [12], # le_session1
    "2019-02-19-10-03-14_1" : [12], # le_session2
    "2019-02-08-11-05-7_2ndPart_1" : [12], # mn_session1
    "2019-02-08-11-12-51_2" : [12, 13], # mn_session1
    "2019-02-15-11-38-22_1" : [12, 13], # mn_session2
    "2019-02-15-11-54-25_2" : [12], # mn_session2
    "2019-01-16-15-18-0_1" : [12], # no_session1
    "2019-01-28-13-31-51_1" : [12], # ph_session1
    "2019-01-28-13-49-14_2" : [12], # ph_session1
    "2019-01-14-15-07-21_1" : [12], # ys_session1
    "2019-01-17-15-05-1_1" : [12], # ys_session3
    "2019-01-30-11-22-25_1" : [12], # ys_session4
    "2019-01-31-13-32-2_2" : [12], # ys_session5
}


# key selection when participants skips some sentences
dict_keySelectionNotCompleted = {
    "2019-01-16-16-36-17_1stPart_2" : [0, 1, 3, 5, 7], # af_session1 ---- last sentence is not finished
    "2019-01-16-17-00-12_2ndPart_2" : [0, 1, 3, 4, 5, 7, 9, 11], # af_session1
    "2019-01-17-15-27-20_1stPart_2" : [0, 1, 3, 5, 7, 9, 11], # af_session2 
    "2019-01-17-16-03-27_2ndPart_2" : [0, 1, 2, 3, 4, 5, 6, 7, 9, 11], # af_session2
    "2019-02-08-11-33-53_1stPart_1" : [0, 1, 3, 4, 5, 7, 9, 11], # aq_session3
    "2019-02-08-12-11-34_2ndPart_1" : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11], # aq_session3
    "2019-01-28-14-30-44_1" : [0, 1, 3, 5], # bh1_session1
    "2019-01-31-09-22-49_1stPart_2": [0, 1, 3, 5, 7, 9, 10, 11], # bh1_session4
    "2019-01-31-09-37-5_2ndPart_2" : [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11], # bh1_session4
    "2019-02-21-16-09-44_1stPart_1" : [0, 1, 3, 4, 5, 7, 9, 11], # bh2_session1
    "2019-02-21-16-22-22_2ndPart_1" : [0, 1, 3, 5, 6, 7, 8, 9, 10, 11], # bh2_session1
    "2019-02-28-17-03-53_1stPart_2" : [0, 1, 3, 5, 6, 7, 9, 11], # bh2_session3
    "2019-02-28-17-24-2_2ndPart_2" : [0, 1, 2, 3, 5], # bh2_session3     ----
    "2019-02-14-13-28-20_1stPart_2" : [0, 1, 3, 5, 6, 7, 9, 11], # cw_session3
    "2019-02-14-13-57-41_2ndPart_2" : [0, 1, 2, 3, 5, 6, 7, 8, 9, 11], # cw_session3
    "2019-02-21-15-01-4_1stPart_1" : [0, 1, 2, 3, 5, 7, 9, 11], # le_session3
    "2019-02-21-15-25-56_2ndPart_1" : [0, 1, 3], # le_session3       ----
    "2019-02-05-14-00-27_1stPart_2" : [0, 1, 3, 5, 7, 8], # mh_session1
    "2019-02-05-14-10-39_2ndPart_2" : [0, 1, 2, 3, 4, 5, 7, 8, 9, 11], # mh_session1
    "2019-02-08-10-51-3_1stPart_1" : [0, 1, 3, 5, 7, 9, 10, 11], # mn_session1
    "2019-02-19-10-34-7_1stPart_1" : [0, 1, 3, 5, 7, 8, 9, 11], # mn_session3
    "2019-02-19-10-56-43_2ndPart_1" : [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11], # mn_session3
    "2019-01-29-13-25-4_1" : [0, 1, 3, 5, 7], # ph_session2  -- sessions where there are less score questions 
    # and more sentences typed
    "2019-01-16-15-18-50_1stPart_1" : [0, 1, 3, 5, 7, 8, 9, 10], # ys_session2
    "2019-01-17-15-05-1_1" : [0, 1, 3, 5],  # ys_session3  -- sessions where there are less score questions 
    # and more sentences typed
    "2019-02-06-11-25-41_1" : [0, 1, 3, 5, 11], # aq_session1 -- sessions where there are less score questions 
    # and more sentences typed
    "2019-01-16-15-42-51_2ndPart_1" : [0, 1, 2, 5], # ys_session2 -- different for reading and writing, this one is for
    # writing
    '2019-01-30-11-22-25_1' : [0, 1, 3, 5, 7, 9, 11],   # ys_session4 -- sessions where there are less score questions 
    # and more sentences typed
    "2019-02-08-11-05-7_2ndPart_1" : [0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11] # mn_session1 -- -- different for reading and 
    # writing, this one is for writing
}

# dictionary for phrase removal just like in the dict_phraseStim, but since not all participants require that, some that 
# do, are added to this new dictionary here
dict_keySelection_phraseStim = {
    '2019-01-17-15-27-20_1stPart_2' : [4], # Af session2
    '2019-01-16-15-18-0_1' : [4],        # no_session1
    '2019-02-19-17-10-45_1' : [3],                  # ph_session5
    '2019-03-07-16-44-5_2' : [1],        # rh_session1
    '2019-03-14-13-56-56_2' : [2],              # rh_session3
    '2019-01-14-15-07-21_1' : [4]         # ys_session1
}


# in the beginning experiments, not everyone started with 800 initial dwell time

dict_dwellTimeOrig_not800 = {
    "2019-01-16-15-51-13_2" : 600, # no_session1
    "2019-01-16-15-18-0_1" : 600, # no_session1
    "2019-01-16-15-43-8_1" : 100, # af_session1
    "2019-01-16-16-36-17_1stPart_2" : 100, # af_session1
    "2019-01-16-17-00-12_2ndPart_2" : 100, # af_session1
    "2019-01-17-15-03-40_1" : 100, # af_session2
    "2019-01-17-15-27-20_1stPart_2" : 0, # af_session2
    "2019-01-17-16-03-27_2ndPart_2" : 100, # af_session2
    "2019-01-14-15-07-21_1" : 500, # ys_session1
    "2019-01-14-15-25-55_2" : 300, # ys_session1
    "2019-01-16-15-18-50_1stpart_1" : 200, # ys_session2
    "2019-01-16-15-42-51_2ndPart_1" : 100, # ys_session2
    "2019-01-16-15-59-55_2" : 100, # ys_session2
    "2019-01-17-15-05-1_1" : 100, # ys_session3
    "2019-01-17-15-31-12_2" : 100 # ys_session3
}


# list of all things that should be present when computing effective time
list_keysToBeCounted = ['Comma', 'BackOne', 'BackMany', 'SpaceBar']

# some sessions do not have gaze data
dict_noGazeData = {
    '2019-01-16-17-00-12_2ndPart_2' : 'no gaze data', # af_session2
    '2019-01-17-15-31-12_2' : 'no gaze data', #ys_session2
    '2019-01-30-11-57-3_2' : 'no gaze data' # ys_session4
}



In [3]:
TimeDwellOrig = 800
TimeFixation = 300

In [4]:
def FixUserKeys(UserKeys_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    Column_beforeDecimal = [item[2] for item in UserKeys_Old]
    Column_afterDecimal = [item[3] if len(item)>3 else '00' for item in UserKeys_Old]
    
    UserKeys_ProgressPercent = [float(Column_beforeDecimal[i]+'.'+ Column_afterDecimal[i]) for i in 
                                range(0, len(Column_beforeDecimal))]
    UserKeys_Times = [item[0] for item in UserKeys_Old]
    UserKeys_Keys = [item[1] for item in UserKeys_Old]
    
    UserKeys_New = [[UserKeys_Times[ind], UserKeys_Keys[ind], UserKeys_ProgressPercent[ind]] for ind in 
                    range(0, len(UserKeys_ProgressPercent))]
    
    #UserKeys_New = np.concatenate((UserKeys_Times, UserKeys_Keys, UserKeys_ProgressPercent), axis = 0)
    
    
    return UserKeys_New
        

In [5]:
def FixScratchPad(ScratchPad_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    ScratchPad_Times = [item[0] for item in ScratchPad_Old]
    
    ScratchPad_Phrases = list()
    
    # loop to combine phrases divided by commas
    ScratchPadInd = -1 
    while ScratchPadInd < len(ScratchPad_Old)-1:
        ScratchPadInd = ScratchPadInd + 1
        commasInPhrase = len(ScratchPad_Old[ScratchPadInd])-2
        if commasInPhrase < 1:
            #print(ScratchPad_Old[ScratchPadInd][1])
            ScratchPad_Phrases.append(ScratchPad_Old[ScratchPadInd][1])
            continue
        scratchPadPhrase = ScratchPad_Old[ScratchPadInd][1]
        for phraseJoinNr in range(1, commasInPhrase+1):
            scratchPadPhrase = scratchPadPhrase + ', ' + ScratchPad_Old[ScratchPadInd][1+phraseJoinNr]
        
        ScratchPad_Phrases.append(scratchPadPhrase)
            
        
    ScratchPad_New = [[ScratchPad_Times[ind], ScratchPad_Phrases[ind]] for ind in 
                    range(0, len(ScratchPad_Times))]
    
    #UserKeys_New = np.concatenate((UserKeys_Times, UserKeys_Keys, UserKeys_ProgressPercent), axis = 0)
    
    #print(ScratchPad_New)
    return ScratchPad_New

In [6]:
def FixKeysSelected(KeysSelected_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    KeysSelected_New = list()
    
    # loop to combine phrases divided by commas
    KeysSelectedInd = -1 
    while KeysSelectedInd < len(KeysSelected_Old)-1:
        KeysSelectedInd = KeysSelectedInd + 1
        
        if KeysSelected_Old[KeysSelectedInd][1].count(',') > 0:
            
            keys_split = KeysSelected_Old[KeysSelectedInd][1].split("\r\n")
            del keys_split[0]
            del keys_split[-1]
            
            keys_split = [key.split(',') for key in keys_split]
            
            KeysSelected_New.extend(keys_split)
        else:
            KeysSelected_New.append(KeysSelected_Old[KeysSelectedInd])
        
    
    return KeysSelected_New

In [7]:
def ComputeDwellTime(userKeys, full_path):
    # modify userKeys to include a column of time instead of progress pct, which is dependent on the then dwell time
    
    TimeDwellOrig = 800
    
    # session name
    session_folder_name = full_path.split('\\')[-1]
    
    if session_folder_name in dic_dwellTimeOrig_not800:
        TimeDwellOrig = dic_dwellTimeOrig_not800[session_folder_name]
    
    #print(TimeDwellOrig)
    
    timeDwell = TimeDwellOrig
    nKey = -1
    for key in userKeys:
        nKey = nKey + 1
        #print(key[1])
        if key[1] == 'IncreaseDwellTime':
            if float(key[2]) == 1:
                timeDwell = timeDwell + 100
        elif key[1] == 'DecreaseDwellTime':
            #print(key[2])
            if float(key[2]) == 1:
                timeDwell = timeDwell - 100
        else:
            userKeys[nKey].append(str(float(key[2])*timeDwell))
    
    return userKeys

In [8]:
def stimPhrasesEdit(PhraseLog, full_path):
   
    # Now extract phrases from the phrase file
    phraseStim_Phrases = [item[1] for item in PhraseLog]
    
    # session name
    session_folder_name = full_path.split('\\')[-1]
        
    phraseStim_PhrasesReduced, phraseStim_timeReduced = zip(*[(x[0], PhraseLog[phraseStim_Phrases.index(x[0])][0]) for x in groupby(phraseStim_Phrases)])
    
    PhraseLogReduced = [[phraseStim_timeReduced[i], phraseStim_PhrasesReduced[i]] for i in range(0, len(phraseStim_PhrasesReduced))]
    
    if PhraseLogReduced[-1][1] == 'THE EXPERIMENT IS NOW DONE':
        del PhraseLogReduced[-1]
        
    if PhraseLogReduced[0][1] == 'phraseText':
        del PhraseLogReduced[0]

    # Here, we want only the sentences typed
    notSentencesToType = list()
    for index in range(0,len(PhraseLogReduced)):
        sentence = PhraseLogReduced[index][1]
        if 'Svar på følgende spørgsmål' in sentence or 'Answer the question:' in sentence or 'What is the complete name of your university?' in sentence or '(give a score between 1 and 7)' in sentence or sentence == '':
            notSentencesToType.append(index)    
    
    for index in sorted(notSentencesToType, reverse=True):
        del PhraseLogReduced[index]
    
    replacingList = []
    PhraseLogReduced = findAndRemoveTrials(session_folder_name, dict_phraseStim, PhraseLogReduced, replacingList)
    
    
    return PhraseLogReduced

In [9]:
# This function will return the datetime in items which is the closest to the date pivot
def nearestTimePoint(dates, date):
    
    for d in dates:
        if d < date:
            nearestTP = d
        else:
            continue
    try: 
        nearestTP
        nearestTPind = dates.index(nearestTP)
    except:
        nearestTP = 0
        nearestTPind = -1
        
    return nearestTP, nearestTPind

In [10]:
def findAndRemoveTrials(session_name, dictionary_saved, trials, replacingList):
    # function to check the session_name in the dictionary_saved and remove those trials from the dictionary_trial
    
    if session_name in dictionary_saved:
        index_list = dictionary_saved[session_name]
    else:
        index_list = replacingList
    
    
    

    if index_list:
        if type(trials) == list:
            for index in sorted(index_list, reverse=True):
                del trials[index]
                
        else:
            for index in sorted(index_list, reverse=True):
                del trials['start'][index]
                del trials['end'][index]
            
    return trials    

In [11]:
# function to convert list of date and time into datetime format list
def timeConversion(timeStrList):
    timeList = list()
    for time in timeStrList:
        time1, t1, t2 = time.partition('+')
        timeList.append(datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
    return timeList

In [12]:
def timeTypingStart(userKeys):
    # From the user keys, find when the user actually starts typing, after having looked at the phrase and all the other 
    # function keys
    
    timeTypingStartInd = 0
    
    timeTypingStartIndList = list()
            
    timeUserTimeInd = 0
    
    ind = 0
    # Get start time of first trial
    
    while ind < len(userKeys):
        #print(len(userKeys[ind][1]))
        if len(userKeys[ind][1]) > 1:
            ind = ind + 1
        else:
            timeTypingStartInd = ind
            timeTypingStartIndList.append(ind)
            break
    
    #print(timeTypingStartInd)
    # Get every next phrase start timings
    while ind < len(userKeys):
        
        if userKeys[ind][1] == 'NextPhrase' and float(userKeys[ind][2]) == 1:
            
            #timeTypingStartIndList.append(ind+1)
            for ind2 in range(ind+1, len(userKeys)):
                if len(userKeys[ind2][1]) > 1:
                    ind = ind + 1
                    continue
                elif userKeys[ind2][1] == 'NextPhrase' and float(userKeys[ind][2]) == 1:
                    ind = ind + 1
                    continue
                else:
                    ind = ind2
                    timeTypingStartIndList.append(ind)
                    break
                    
        else:
            ind = ind + 1
            
    #print(timeTypingStartIndList)
    
    return timeTypingStartIndList

In [13]:
def OptiKeyTypingTime(UserKeys):
    
    TimeTyping = dict()
    
    time1, t1, t2 = UserKeys[0][0].partition('+')
    startTime = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
    time2, t1, t2 = UserKeys[-1][0].partition('+')
    endTime = datetime.datetime.strptime(re.sub('[:.T]','-',time2[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
    TimeTyping['startTime'] = startTime
    TimeTyping['endTime'] = endTime
    
    return TimeTyping

In [32]:
def FindTrialTimes(KeysSelected, timeTyping, full_path):
    # function to find start and end of tasks in experiments
    
    # session name
    session_folder_name = full_path.split('\\')[-1]
    
    timeTrialDict = dict()
    timeTrialDict = {'start': [],
                    'end':[]}
    
    nTrial = -1
    
    for keys in KeysSelected:
        
            
        
        if keys[1] == 'NextPhrase':
            nTrial = nTrial + 1
            time1, t1, t2 = keys[0].partition('+')
            endTimeTrial = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
            
            if nTrial != 0:
                # print('end: ', endTimeTrial)
                #print('')
                timeTrialDict['end'].append(endTimeTrial)
            
            
            # add 5s for the start time of the next phrase
            seconds_start = keys[0][17:19]
            
            if int(seconds_start) > 54:
                minute_start = keys[0][14:16]
                seconds_start_new = str(int(seconds_start) - 55)
            
                if int(minute_start) > 58:
                    minute_start_new = str(int(minute_start) - 59)
                    hour_start_new = str(int(keys[0][11:13]) + 1)
                        
                else:
                    minute_start_new = str(int(minute_start) + 1)
                    hour_start_new = str(int(keys[0][11:13]))
                        
            else:
                seconds_start_new = str(int(seconds_start) + 5)
                minute_start_new = str(keys[0][14:16])
                hour_start_new = str(int(keys[0][11:13]))
                    
            endTimew5s = keys[0][0:11] + hour_start_new + ':' + minute_start_new + ':' + seconds_start_new + keys[0][19:]
            
            time1, t1, t2 = endTimew5s.partition('+')
            startTimeTrial = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
            
            #print('start: ', startTimeTrial)
            timeTrialDict['start'].append(startTimeTrial)
        
    del timeTrialDict['start'][-1]
    
    # remove the extra selections of NewPhrase at the end of some sessions
    replacingList = []
    timeTrialDict = findAndRemoveTrials(session_folder_name, dict_keySelectionOfNextPhrase, timeTrialDict, replacingList)
    
    
    scoreQuestions = [0, 1, 3, 5, 7, 9, 11]
    timeTrialDict = findAndRemoveTrials(session_folder_name, dict_keySelectionNotCompleted, timeTrialDict, scoreQuestions)
    
    timeTrialDict = findAndRemoveTrials(session_folder_name, dict_keySelection_phraseStim, timeTrialDict, replacingList)
    
    
    #for i in timeTrialDict['end']:
    #    print(i)
        
    return timeTrialDict

In [33]:
def ExtractScores(EventTrials_fromKeySelection, KeysSelected, full_path):
    
    # session name
    session_folder_name = full_path.split('\\')[-1]
    
    # KeysSelected timings in the correct format
    KeysSelected_timeStr = [key[0] for key in KeysSelected]
    KeysSelected_time = timeConversion(KeysSelected_timeStr)
    
    KeysSelected_keys = [key[1] for key in KeysSelected]
    
    ScoreDifficultyList = list()
    
    # use the trial ending time, and extract the number key selected between 2 trials
    for trialEndInd, trialEnd in enumerate(EventTrials_fromKeySelection['end']):
        
        trialEnd_keySelectionTimePrevious, trialEnd_keySelectionInd = nearestTimePoint(KeysSelected_time, trialEnd)
        trialEnd_keySelectionTime = KeysSelected_time[trialEnd_keySelectionInd+1]
        #print('start', trialEnd_keySelectionTime)
        
        if trialEndInd < len(EventTrials_fromKeySelection['end'])-1: # as long as it's not the last trial
            trialNextStart = EventTrials_fromKeySelection['start'][trialEndInd+1]
            trialNextStart_keySelectionTimePrevious, trialNextStart_keySelectionInd = nearestTimePoint(KeysSelected_time, trialNextStart)
            trialNextStart_keySelectionTime = KeysSelected_time[trialNextStart_keySelectionInd+1]
            
            
        else:
            
            # find if the NextPhrase key is pressed again
            NextPhrase_afterLastScoreQuestion = [(ind, KeysSelected_time[trialEnd_keySelectionInd+ind+1]) for ind, key in enumerate(KeysSelected_keys[trialEnd_keySelectionInd+1:]) if key == 'NextPhrase']
            #print('last next phrase selected: ', NextPhrase_afterLastScoreQuestion)
            
            if not NextPhrase_afterLastScoreQuestion:
                trialNextStart_keySelectionTime = KeysSelected_time[-1]
                trialNextStart_keySelectionInd = len(KeysSelected_time)-1
            else:
                if len(NextPhrase_afterLastScoreQuestion) > 1:
                    NextPhrase_ind = 1
                else:
                    NextPhrase_ind = 0 
                
                trialNextStart_keySelectionTime = NextPhrase_afterLastScoreQuestion[NextPhrase_ind][1]
                trialNextStart_keySelectionInd = trialEnd_keySelectionInd + 1 + NextPhrase_afterLastScoreQuestion[NextPhrase_ind][0]

            #print(KeysSelected_keys[trialNextStart_keySelectionInd])
        
        scoreTrialKeys = KeysSelected_keys[trialEnd_keySelectionInd+1:trialNextStart_keySelectionInd+1]
        score = [key for key in scoreTrialKeys if key.isdigit()]
        
        if '0' in score:
            del score[score.index('0')]
        
        #print('end', trialNextStart_keySelectionTime) 
        if not score:
            ScoreDifficultyList.append(np.nan)
        else:
            ScoreDifficultyList.append(int(score[-1]))
            
            
        #print(ScoreDifficultyList[-1])
        #print('')
        
    return ScoreDifficultyList
    

In [34]:
class DataForEveryTrial:
    subjectID = ''
    blockNumber = ''
    sessionNumber = ''
    nTrials = ''
    variable = ''
    dataForTrial = ''
    resultPathName = ''
   
    
    def printInfo(self):
        dataFrame = pd.DataFrame(list(zip([self.subjectID]*len(self.dataForTrial), [self.blockNumber]*len(self.dataForTrial), [self.sessionNumber]*len(self.dataForTrial), range(0,len(self.dataForTrial)+1), self.dataForTrial)), columns=['subjectID', 'block', 'session', 'trial', self.variable])
        
        return dataFrame
    
    def AddToFile(self):
        
        dataFrame = pd.DataFrame(list(zip([self.subjectID]*len(self.dataForTrial), [self.blockNumber]*len(self.dataForTrial), [self.sessionNumber]*len(self.dataForTrial), range(0,len(self.dataForTrial)+1), self.dataForTrial)), columns=['subjectID', 'block', 'session', 'trial', self.variable])
        book = load_workbook(self.resultPathName)
        writer = pd.ExcelWriter(self.resultPathName, engine='openpyxl')
        writer.book = book
        writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
        startrow = writer.sheets['Sheet1'].max_row
        dataFrame.to_excel(writer, startrow = startrow, index = False, header = False)
        
        writer.save()
        
    
    

In [39]:
metricComputed = 'scoreDifficulty'
print(metricComputed)

dataFolderName = r'E:\Data\Data' # accessing external hard disk with the data
a = re.compile('(?<=Data\\\\Data\\\\)(.*)(?=\\\\[1-9])')
subjectName_listElement = 3

#dataFolderName = r'C:\DTU\Data\201901_JanuaryExpt' # accessing data saved in the computer
#a = re.compile('(?<=Data\\\\201901_JanuaryExpt\\\\)(.*)(?=\\\\[1-9])')
#subjectName_listElement = 4

resultFileName = r'C:\DTU\Data\201901_JanuaryExpt\DataExtracted\Subject_Block_Session_Trial_' + metricComputed +  '.xlsx'

for root, dirs, subfolder in os.walk(dataFolderName):
    
    technique = 'dwell_time'
    
    if not dirs:
        
        #if 'notCompleted' in root or 'notInclude' in root: # Some subjects do not have gaze log and have been marked as 
        
        if 'noData' in root or 'Trial' in root or 'trial' in root or 'Nothing' in root: # Some subjects do not have gaze log and have been marked as 
            #notInclude
            continue
        if 'Jonas' in root or 'Praktikant' in root or 'Villads' in root:
            continue
            
        #if 'af\\1\\' not in root:
        #    continue
        if 'Picture' in root:
            continue
        #if '_MS' not in root:
        #    continue
        
        
        
        
        #if '2019-1-16-16-36-17_1stPart_2' not in root:
        #    continue
            
        keysSelected = None
        userKeys = None
        phraseLog = None
        
        for file in subfolder:
            
            
            if fnmatch.fnmatch(file, 'KeySelection*'):
                try:
                    
                    fKeysSelected = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerKeysSelected = csv.reader(fKeysSelected)
                    keysSelected = list(readerKeysSelected)
                    
                    keysSelected.remove(keysSelected[0])
                except:
                    if fKeysSelected is not None:
                        fKeysSelected.close()
                    else:
                        print('error in opening the KeySelection log file')
                        
            
            if fnmatch.fnmatch(file, 'user*'):
                try:
                    fUserKey = open(root + '\\' + file, encoding='utf-8',  newline='')
                    readerUserKey = csv.reader(fUserKey, quotechar=None)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user key log file')
                        
            if fnmatch.fnmatch(file, 'phrase*'):
                try:
                    fPhraseLog = open(root + '\\' + file, encoding='utf-8')
                    readerPhraseLog = csv.reader(fPhraseLog, quotechar=None)
                    phraseLog = list(readerPhraseLog)
                    
                except:
                    if fPhraseLog is not None:
                        fPhraseLog.close()
                    else:
                        print('error in opening the phrase log file')
                        
            if fnmatch.fnmatch(file, 'multiKey*'):
                technique = 'multiKey_selection'
            
            if fnmatch.fnmatch(file, 'tobiiGazeLog*'):
                try:
                    fGazeLog = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerGazeLog = csv.reader(fGazeLog, quotechar=None)
                    gazeLog = list(readerGazeLog)
                    gazeLog.remove(gazeLog[0])
                    gazeLog.remove(gazeLog[-1])
                except:
                    if fGazeLog is not None:
                        fGazeLog.close()
                    else:
                        print('error in opening the scratchpad log file')
            if fnmatch.fnmatch(file, 'ScratchPadLog*'):
                try:
                    fScratchPad = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerScratchPad = csv.reader(fScratchPad, quotechar=None)
                    scratchPad = list(readerScratchPad)
                except:
                    if fScratchPad is not None:
                        fScratchPad.close()
                    else:
                        print('error in opening the scratchpad log file')
            
                    
                     
        if keysSelected is None or userKeys is None or phraseLog is None or gazeLog is None or scratchPad is None:
            continue
        else:
                
            #print('subject path', root)
            subjAndSessionName = '__'.join(root.split('\\')[subjectName_listElement:])
            subjName = subjAndSessionName.split('__')[0]
            print('subject and session name: ', subjAndSessionName)
            sessionFolderName = root.split('\\')[-1]
            
            # fix phraselog due to comma related file changes
            phraseLog_new = FixScratchPad(phraseLog)
            
            # fix userKeys due to comma related file changes
            userKeys_new = FixUserKeys(userKeys)
            
            
            # need to fix keys selected, where rows get combined because of an inverted comma
            keysSelected_new = FixKeysSelected(keysSelected)
            
            # find start time of typing
            timeTyping = OptiKeyTypingTime(userKeys_new)
            
            # divide complete data into epochs of phrases
            timeStartEndDict = FindTrialTimes(keysSelected_new, timeTyping, root)
            
            
            # extract subjective scores after each trial
            scoreExpected = ExtractScores(timeStartEndDict, keysSelected_new, root)
            
            if '1stPart' in root:
                scoreExpected1 = scoreExpected
                continue
                
            if '2ndPart' in root:
                scoreExpected2 = scoreExpected
                
                scoreExpected = scoreExpected1 + scoreExpected2
                scoreExpected1 = list()
                
            
            dataToSave = DataForEveryTrial()
            dataToSave.subjectID = subjAndSessionName.split('__')[0]
            dataToSave.blockNumber = subjAndSessionName.split('__')[1]
            dataToSave.sessionNumber = subjAndSessionName[-1]
            dataToSave.variable = metricComputed
            dataToSave.dataForTrial = scoreExpected
            dataToSave.nTrials = len(scoreExpected)
            dataToSave.resultPathName = resultFileName
            
            print(dataToSave.printInfo())
            dataToSave.AddToFile()
             

scoreDifficulty
subject and session name:  ac__1__2019-02-11-11-18-30_1
  subjectID block session  trial  scoreDifficulty
0        ac     1       1      0                4
1        ac     1       1      1                3
2        ac     1       1      2                3
3        ac     1       1      3                4
4        ac     1       1      4                4
subject and session name:  ac__1__2019-02-11-11-33-32_2
  subjectID block session  trial  scoreDifficulty
0        ac     1       2      0                5
1        ac     1       2      1                6
2        ac     1       2      2                6
3        ac     1       2      3                6
4        ac     1       2      4                6
subject and session name:  ac__2__2019-02-12-15-00-4_1
  subjectID block session  trial  scoreDifficulty
0        ac     2       1      0                5
1        ac     2       1      1                3
2        ac     2       1      2                4
3        ac     2

subject and session name:  aq__2__2019-02-07-11-01-57_1
  subjectID block session  trial  scoreDifficulty
0        aq     2       1      0                2
1        aq     2       1      1                4
2        aq     2       1      2                5
3        aq     2       1      3                6
4        aq     2       1      4                2
subject and session name:  aq__2__2019-02-07-11-24-49_2
  subjectID block session  trial  scoreDifficulty
0        aq     2       2      0                4
1        aq     2       2      1                1
2        aq     2       2      2                6
3        aq     2       2      3                5
4        aq     2       2      4                5
subject and session name:  aq__3__2019-02-08-11-33-53_1stPart_1
subject and session name:  aq__3__2019-02-08-12-11-34_2ndPart_1
  subjectID block session  trial  scoreDifficulty
0        aq     3       1      0                6
1        aq     3       1      1                6
2        a

subject and session name:  bh2__3__2019-02-28-16-47-35_1
  subjectID block session  trial  scoreDifficulty
0       bh2     3       1      0                2
1       bh2     3       1      1                1
2       bh2     3       1      2                2
3       bh2     3       1      3                4
4       bh2     3       1      4                3
subject and session name:  bh2__3__2019-02-28-17-03-53_1stPart_2
subject and session name:  bh2__3__2019-02-28-17-24-2_2ndPart_2
  subjectID block session  trial  scoreDifficulty
0       bh2     3       2      0                4
1       bh2     3       2      1                5
2       bh2     3       2      2                4
3       bh2     3       2      3                3
4       bh2     3       2      4                2
subject and session name:  bh2__4__2019-03-04-10-14-44_1
  subjectID block session  trial  scoreDifficulty
0       bh2     4       1      0                3
1       bh2     4       1      1                3
2      

subject and session name:  jm__4__2019-02-05-15-01-43_2
  subjectID block session  trial  scoreDifficulty
0        jm     4       2      0                2
1        jm     4       2      1                4
2        jm     4       2      2                3
3        jm     4       2      3                4
4        jm     4       2      4                6
subject and session name:  jm__5__2019-02-06-15-32-48_1
  subjectID block session  trial  scoreDifficulty
0        jm     5       1      0                5
1        jm     5       1      1                5
2        jm     5       1      2                4
3        jm     5       1      3                5
4        jm     5       1      4                6
subject and session name:  jm__5__2019-02-06-16-10-7_2
  subjectID block session  trial  scoreDifficulty
0        jm     5       2      0                1
1        jm     5       2      1                2
2        jm     5       2      2                1
3        jm     5       2      3 

subject and session name:  le__1__2019-02-18-13-20-38_1
  subjectID block session  trial  scoreDifficulty
0        le     1       1      0                5
1        le     1       1      1                4
2        le     1       1      2                4
3        le     1       1      3                2
4        le     1       1      4                3
subject and session name:  le__1__2019-02-18-14-02-56_2
  subjectID block session  trial  scoreDifficulty
0        le     1       2      0                4
1        le     1       2      1                2
2        le     1       2      2                2
3        le     1       2      3                2
4        le     1       2      4                5
subject and session name:  le__2__2019-02-19-10-03-14_1
  subjectID block session  trial  scoreDifficulty
0        le     2       1      0                5
1        le     2       1      1                4
2        le     2       1      2                4
3        le     2       1      3

subject and session name:  ls2__2__2019-02-12-10-23-24_2
  subjectID block session  trial  scoreDifficulty
0       ls2     2       2      0                6
1       ls2     2       2      1                7
2       ls2     2       2      2                7
3       ls2     2       2      3                2
4       ls2     2       2      4                7
subject and session name:  ls2__3_MS__2019-02-13-15-20-38_1
  subjectID block session  trial  scoreDifficulty
0       ls2  3_MS       1      0                6
1       ls2  3_MS       1      1                7
2       ls2  3_MS       1      2                7
3       ls2  3_MS       1      3                6
4       ls2  3_MS       1      4                6
subject and session name:  ls2__3_MS__2019-02-13-15-55-28_2
  subjectID block session  trial  scoreDifficulty
0       ls2  3_MS       2      0                4
1       ls2  3_MS       2      1                5
2       ls2  3_MS       2      2                6
3       ls2  3_MS      

subject and session name:  mn__4_MS__2019-02-21-17-26-10_1
  subjectID block session  trial  scoreDifficulty
0        mn  4_MS       1      0                4
1        mn  4_MS       1      1                7
2        mn  4_MS       1      2                4
3        mn  4_MS       1      3                5
4        mn  4_MS       1      4                5
subject and session name:  mn__4_MS__2019-02-21-17-45-11_2
  subjectID block session  trial  scoreDifficulty
0        mn  4_MS       2      0                5
1        mn  4_MS       2      1                5
2        mn  4_MS       2      2                7
3        mn  4_MS       2      3                6
4        mn  4_MS       2      4                7
subject and session name:  mn__5__2019-02-22-10-41-37_1
  subjectID block session  trial  scoreDifficulty
0        mn     5       1      0                4
1        mn     5       1      1                2
2        mn     5       1      2                2
3        mn     5       1 

subject and session name:  ph__5__2019-02-19-17-24-20_2
  subjectID block session  trial  scoreDifficulty
0        ph     5       2      0                4
1        ph     5       2      1                5
2        ph     5       2      2                4
3        ph     5       2      3                5
4        ph     5       2      4                7
subject and session name:  rh__1__2019-03-07-16-17-30_1
  subjectID block session  trial  scoreDifficulty
0        rh     1       1      0                3
1        rh     1       1      1                5
2        rh     1       1      2                4
3        rh     1       1      3                6
4        rh     1       1      4                6
subject and session name:  rh__1__2019-03-07-16-44-5_2
  subjectID block session  trial  scoreDifficulty
0        rh     1       2      0                6
1        rh     1       2      1                6
2        rh     1       2      2                8
3        rh     1       2      3 

In [ ]:
a = [5]
print(a[-1])

In [ ]:
def PlotPupilSizeWithEvents_(PupilData_df, EventsTrials, PlotTitle, SubjectAndSessionName):
    
    subjectID = SubjectAndSessionName.split('__')[0]
    
    EventTrials_index = dict()
    EventTrials_index['start'] = [i for i, x in enumerate(EventsTrials['start']) if x] 
    EventTrials_index['end'] = [i for i, x in enumerate(EventsTrials['end']) if x] 
    
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    
    ax.plot(PupilData_df['timeStamp'], PupilData_df['pupilLeft'], 'b', label = 'Left')
    ax.plot(PupilData_df['timeStamp'], PupilData_df['pupilRight'], 'r', label = 'Right')
    
    for start in EventTrials_index['start']:
        ax.plot([PupilData_df['timeStamp'][start], PupilData_df['timeStamp'][start]], [np.min(PupilData_df['pupilLeft']), np.max(PupilData_df['pupilLeft'])], color = 'orange')
    
    for end in EventTrials_index['end']:
        ax.plot([PupilData_df['timeStamp'][end], PupilData_df['timeStamp'][end]], [np.min(PupilData_df['pupilLeft']), np.max(PupilData_df['pupilLeft'])], color = 'green')
    
    ax.set_ylabel('Absolute pupil size [in mm]')
    ax.set_xlabel('Time [in s]')
    plt.legend()
    plt.title('plot {}  for subject  {}'.format(PlotTitle, SubjectAndSessionName))
    
    #pickle.dump(fig, open(r'C:\DTU\Results\201901_Expt\Expt_wTrialsMarked' + '\\' + subjectID + '\\' + 'pupilSize_CompleteExperiment_w' + PlotTitle + SubjectAndSessionName, 'wb'))

    
